In [3]:
import numpy as np
import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
import re
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
from sklearn.metrics import classification_report

In [4]:
data=pd.read_csv("spam.csv",encoding='latin-1')
data

,v1,v2,Unnamed: 2,Unnamed: 3,Unnamed: 4
0,ham,"Go until jurong point, crazy.. Available only ...",NaN,NaN,NaN
1,ham,Ok lar... Joking wif u oni...,NaN,NaN,NaN
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,NaN,NaN,NaN
3,ham,U dun say so early hor... U c already then say...,NaN,NaN,NaN
4,ham,"Nah I don't think he goes to usf, he lives aro...",NaN,NaN,NaN
...,...,...,...,...,...
5567,spam,This is the 2nd time we have tried 2 contact u...,NaN,NaN,NaN
5568,ham,Will Ì_ b going to esplanade fr home?,NaN,NaN,NaN
5569,ham,"Pity, * was in mood for that. So...any other s...",NaN,NaN,NaN
5570,ham,The guy did some bitching but I acted like i'd...,NaN,NaN,NaN


In [5]:
#drop 'Unnamed: 2','Unnamed: 3','Unnamed: 4'
data = data.drop(['Unnamed: 2','Unnamed: 3','Unnamed: 4'],axis=1)
data.head()


,v1,v2
0,ham,"Go until jurong point, crazy.. Available only ..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives aro..."


# Count the number of words in each Text

In [6]:
data['Count']=0
for i in np.arange(0,len(data.v2)):
    data.loc[i,'Count'] = len(data.loc[i,'v2'])

In [7]:
data.head()

,v1,v2,Count
0,ham,"Go until jurong point, crazy.. Available only ...",111
1,ham,Ok lar... Joking wif u oni...,29
2,spam,Free entry in 2 a wkly comp to win FA Cup fina...,155
3,ham,U dun say so early hor... U c already then say...,49
4,ham,"Nah I don't think he goes to usf, he lives aro...",61


In [8]:
# Total ham(0) and spam(1) messages
data['v1'].value_counts()

v1
ham     4825
spam     747
Name: count, dtype: int64

In [9]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 3 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
 2   Count   5572 non-null   int64 
dtypes: int64(1), object(2)
memory usage: 130.7+ KB


In [14]:
stored_List_afterprocessing=[]
porter_stem=PorterStemmer()
print(data['v2'][0])
print(data['v2'][1])

Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there got amore wat...
Ok lar... Joking wif u oni...


In [15]:
for i in range(0, 5572):
    # Apply Regular Expression
    msg = data['v2'][i]

    # Replace email addresses with 'emailaddr'
    msg = re.sub(r'\b\w+@\w+\.\w{2,4}\b', 'emailaddr', msg)

    # Replace URLs with 'httpaddr'
    msg = re.sub(r'http[s]?:\/\/\S+', 'httpaddr', msg)

    # Replace money symbols with 'moneysymb'
    msg = re.sub(r'[£$]', 'moneysymb', msg)

    # Replace phone numbers with 'phonenumbr'
    msg = re.sub(r'\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b', 'phonenumbr', msg)

    # Replace numbers with 'numbr'
    msg = re.sub(r'\b\d+(\.\d+)?\b', 'numbr', msg)

    # Remove all punctuations
    msg = re.sub('[^\w\d\s]', '', data['v2'][i])

    if i < 2:  # if i < 2 is used in the code to print out the intermediate results for only the first two iterations of the loop
        print("\t\t\t\t MESSAGE", i)
    if i < 2:
        print("\n After Regular Expression - Message", i, ":", msg)

    ########################################################3
    # Other processes without regular expression
    msg = msg.lower()
    if i < 2:
        print("\n Lower Case Message", i, ":", msg)
    # Splitting words to Tokenize
    msg = msg.split()
    if i < 2:
        print("\n After Splitting - Message", i, ":", msg)
    # Stemming with PorterStemmer handling Stop Words
    msg = [porter_stem.stem(word) for word in msg if not word in set(stopwords.words("english"))]
    if i < 2:
        print("\n After Stemming - Message ", i, " : ", msg)
    # Preparing Messages with Remaining Tokens
    msg = ' '.join(msg)
    if i < 2:
        print("\n Final Prepared - Message ", i, " : ", msg, "\n\n")

    # Preparing WordVector Corpus
    stored_List_afterprocessing.append(msg)


<>:21: SyntaxWarning: invalid escape sequence '\w'
<>:21: SyntaxWarning: invalid escape sequence '\w'
C:\Users\lenovo\AppData\Local\Temp\ipykernel_21924\4147524186.py:21: SyntaxWarning: invalid escape sequence '\w'
  msg = re.sub('[^\w\d\s]', '', data['v2'][i])


				 MESSAGE 0

 After Regular Expression - Message 0 : Go until jurong point crazy Available only in bugis n great world la e buffet Cine there got amore wat

 Lower Case Message 0 : go until jurong point crazy available only in bugis n great world la e buffet cine there got amore wat

 After Splitting - Message 0 : ['go', 'until', 'jurong', 'point', 'crazy', 'available', 'only', 'in', 'bugis', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'there', 'got', 'amore', 'wat']

 After Stemming - Message  0  :  ['go', 'jurong', 'point', 'crazi', 'avail', 'bugi', 'n', 'great', 'world', 'la', 'e', 'buffet', 'cine', 'got', 'amor', 'wat']

 Final Prepared - Message  0  :  go jurong point crazi avail bugi n great world la e buffet cine got amor wat 


				 MESSAGE 1

 After Regular Expression - Message 1 : Ok lar Joking wif u oni

 Lower Case Message 1 : ok lar joking wif u oni

 After Splitting - Message 1 : ['ok', 'lar', 'joking', 'wif', 'u', 'oni']

 After Stemming - Message  1  :  ['ok'

'''
\w+@\w+\.\w{2,4} is used to match email addresses.
http[s]?:\/\/\S+ is used to match URLs.
[£$] is used to match money symbols.
\b(\+\d{1,2}\s)?\d?[\-(.]?\d{3}\)?[\s.-]?\d{3}[\s.-]?\d{4}\b is used to match phone numbers.
\b\d+(\.\d+)?\b is used to match numbers.
\b matches at the beginning or end of a word
For example:
\bword\b matches the word "word" only if it appears as a whole word, not as part of another word like "keyword" or "words".
\b\d{3}\b matches a three-digit number only if it appears as a whole word, not as part of a longer number like "1234" or "456789"
.
'''

In [17]:
countVec=CountVectorizer()#CountVectorizer() is a class from the scikit-learn library used to convert a collection of text documents into a matrix of token counts.
x=countVec.fit_transform(stored_List_afterprocessing).toarray()

In [19]:
#Classification
# i will work with v1 
y=data['v1']
print(y.value_counts())
print(y[0])
print(y[1])

v1
ham     4825
spam     747
Name: count, dtype: int64
ham
ham


In [20]:
### Encoding Labels i want tp
encoding=LabelEncoder()#This class is used to encode categorical target labels with integer values
y=encoding.fit_transform(y)#fit(): It learns the vocabulary dictionary of the input data 
print(y)
print(y[0])
print(y[1])

[0 0 1 ... 0 0 0]
0
0


# This class is used to encode categorical target labels with integer values


In [21]:
 xtrain,xtest,ytrain,ytest=train_test_split(x,y,test_size=0.20,random_state=0)
    

# Applying Guassian Naive Bayes
#is one of the best algorithms used with text analysis)


In [23]:
bayes_classifier=GaussianNB()
bayes_classifier.fit(xtrain,ytrain)##fit(): It learns the vocabulary dictionary of the input data 


GaussianNB()

In [24]:
#Predicting
y_pred=bayes_classifier.predict(xtest)
y_pred


array([0, 0, 0, ..., 0, 0, 0])

# Results

In [25]:
# Evaluating
cMatrix=confusion_matrix(ytest,y_pred)#confusion_matrix function is a tool used to evaluate the performance of a classification model. It compares the actual labels of the test data with the predicted labels generated by the model
cMatrix

array([[816, 133],
       [ 26, 140]], dtype=int64)

The top-left value (816) represents the number of true negatives (TN).
The top-right value (133) represents the number of false positives (FP).
The bottom-left value (26) represents the number of false negatives (FN).
The bottom-right value (140) represents the number of true positives (TP).

In [26]:

# Calculating and printing the accuracy score
accuracy = accuracy_score(ytest, bayes_classifier.predict(xtest))
print("Accuracy :%0.5f" % accuracy)

# Generating and printing the classification report
report = classification_report(ytest, bayes_classifier.predict(xtest))
print(report)


Accuracy :0.85740
              precision    recall  f1-score   support

           0       0.97      0.86      0.91       949
           1       0.51      0.84      0.64       166

    accuracy                           0.86      1115
   macro avg       0.74      0.85      0.77      1115
weighted avg       0.90      0.86      0.87      1115



# Applying Decision Tree


In [27]:
dTree=DecisionTreeClassifier(random_state=50)
dTree.fit(xtrain,ytrain)

DecisionTreeClassifier(random_state=50)

In [28]:
#predicting
y_pred_dTree=dTree.predict(xtest)

In [29]:
#Evaluating
CMatrix=confusion_matrix(ytest,y_pred_dTree)
CMatrix

array([[943,   6],
       [ 34, 132]], dtype=int64)

In [30]:
print ("Accuracy : %0.5f \n\n" % accuracy_score(ytest, dTree.predict(xtest)))
print (classification_report(ytest, dTree.predict(xtest)))

Accuracy : 0.96413 


              precision    recall  f1-score   support

           0       0.97      0.99      0.98       949
           1       0.96      0.80      0.87       166

    accuracy                           0.96      1115
   macro avg       0.96      0.89      0.92      1115
weighted avg       0.96      0.96      0.96      1115

